In [57]:
import pandas as pd
import pathpy as pp
import numpy as np


In [5]:
def load_df():
    network_df = pd.read_excel (r'manufacturing_emails_temporal_network.xlsx')
    return network_df

def load_df_as_network(df):
    temporal = pp.TemporalNetwork()
    for ind in df.index:
        temporal.add_edge(df['node1'][ind], df['node2'][ind], int(df['timestamp'][ind]))
    return temporal

net_df = load_df()
temporal = load_df_as_network(net_df)


In [75]:
links = net_df[["node1","node2"]]
links = links.drop_duplicates()
net = pp.Network(directed=False)
np_links = links.to_numpy()
for l in np_links:
    net.add_edge(l[0]-1, l[1]-1)
print(net)


Undirected network
Nodes:				167
Links:				3250



In [73]:
def q(network, C=None, delta=None):
    assert C is None or delta is None, 'Error: Cannot use clustering and delta-function simultaneously'

    m = network.ecount()
    A = network.adjacency_matrix(weighted=False)
    idx = network.node_to_name_map()
    q = 0.0
    print(network.nodes[1])
    for v in range(1, len(network.nodes)):
        print(network.nodes[v])
        for w in range(1, len(network.nodes)):
            if (C != None and C[v] == C[w]) or (delta != None and delta(v,w)):
                print(v)
                q += A[idx[v], idx[w]] - network.nodes[v]['indegree']*network.nodes[w]['outdegree']/(2*m)
    q /= 2*m
    return q

def q_merge(network, C, merge=None):
    m = network.ecount()
    n = network.ncount()    
    A = network.adjacency_matrix(weighted=False)
    idx = network.node_to_name_map()
    q = 0.0
    for v in network.nodes:
        for w in network.nodes:
            if C[v] == C[w] or (merge is not None and C[v] in merge and C[w] in merge):
                q += A[idx[v], idx[w]] - network.nodes[v]['indegree']*network.nodes[w]['outdegree']/(2*m)
    q /= 2*m
    return q

def q_max(network, C=None, delta=None):
    assert C is None or delta is None, 'Error: Cannot use clustering and delta-function simultaneously'

    m = network.ecount()
    idx = network.node_to_name_map()
    q = 0.0
    for v in network.nodes:
        for w in network.nodes:
            if (C != None and C[v] == C[w]) or (delta != None and delta(v,w)):
                q -= network.nodes[v]['indegree']*network.nodes[w]['outdegree']/(2*m)
    q /= 2*m
    return q

In [55]:
def find_communities(network, iterations=100):
    # start with each node being in a separate cluster
    C = {}
    community_to_nodes = {}
    c = 0
    for n in network.nodes:
        C[n] = c
        community_to_nodes[c] = set([n])
        c += 1

    print("hghhv")
    print(network.nodes[6])
    q_current = q(network, C)
    communities = list(C.values())
    
    for i in range(iterations):
        # randomly choose two communities
        x, y = np.random.choice(communities, size=2)
        # check Q of merged communities
        q_new = q_merge(network, C, merge=set([x, y]))
        if q_new > q_current:
            # actually merge the communities
            for n in community_to_nodes[x]:
                C[n] = y
            community_to_nodes[y] = community_to_nodes[y] | community_to_nodes[x]
            q_current = q_new
            communities.remove(x)
            del community_to_nodes[x]
    return C, q_current

In [37]:
def assortativity_coeff(network, C=None):
    C, q_opt = find_communities(network)
    return q_opt/q_max(network, C)

In [77]:
pp.algorithms.modularity.assortativity_coeff(net)
# assortativity_coeff(net)

0.37010452589087517